In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.sample(5)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
48761,29794,Operations,region_13,Bachelor's,m,other,1,31,3.0,2,0,0,59,0
37978,57756,Analytics,region_28,Master's & above,m,sourcing,2,37,3.0,5,1,1,83,1
30140,1991,Operations,region_4,Master's & above,f,sourcing,1,43,5.0,4,0,0,77,1
22783,68459,Finance,region_2,Master's & above,m,other,1,47,3.0,18,0,0,65,0
19409,24728,Analytics,region_22,Master's & above,m,sourcing,1,36,4.0,9,0,0,82,0


In [4]:
df.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [5]:
df['education'] = df['education'].fillna(df['education'].mode()[0])

In [6]:
df['previous_year_rating'] = df['previous_year_rating'].fillna(0)

In [7]:
df.isnull().sum()

employee_id             0
department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
is_promoted             0
dtype: int64

In [8]:
df.shape

(54808, 14)

In [21]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False, drop='first') 

In [22]:
categorical_columns = df.select_dtypes(include=['object']).columns
onehot_encoded = onehot_encoder.fit_transform(df[categorical_columns])
onehot_encoded_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))
df_encoded = pd.concat([df.drop(columns=categorical_columns), onehot_encoded_df], axis=1)

C:\Users\mymik\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [23]:
x = df_encoded.drop(['is_promoted'], axis=1)
y = df_encoded['is_promoted']

In [24]:
x.shape

(54808, 54)

In [25]:
y.shape

(54808,)

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [27]:
clf = DecisionTreeClassifier(random_state=42)

In [28]:
clf.fit(x, y)

DecisionTreeClassifier(random_state=42)

In [29]:
X_test = pd.read_csv('test.csv')

In [35]:
X_test.shape

(23490, 13)

In [30]:
X_test.isnull().sum()

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [31]:
X_test['education'] = X_test['education'].fillna(X_test['education'].mode()[0])
X_test['previous_year_rating'] = X_test['previous_year_rating'].fillna(0)

In [32]:
X_test.isnull().sum()

employee_id             0
department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
dtype: int64

In [33]:
categorical_columns_test = X_test.select_dtypes(include=['object']).columns
onehot_encoded_test = onehot_encoder.fit_transform(X_test[categorical_columns])
onehot_encoded_X_test = pd.DataFrame(onehot_encoded_test, columns=onehot_encoder.get_feature_names_out(categorical_columns_test))
test_encoded = pd.concat([X_test.drop(columns=categorical_columns_test), onehot_encoded_X_test], axis=1)

C:\Users\mymik\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [34]:
test_encoded.shape

(23490, 54)

In [36]:
y_pred = clf.predict(test_encoded)

In [37]:
type(y_pred)

numpy.ndarray

In [38]:
Y_pred = pd.DataFrame(y_pred, columns=['is_promoted'])

In [39]:
type(Y_pred)

pandas.core.frame.DataFrame

In [47]:
y_test = pd.read_csv('sample_submission.csv')

In [51]:
type(y_test)

pandas.core.frame.DataFrame

In [52]:
accuracy = accuracy_score(Y_pred, y_test['is_promoted']) * 100

In [53]:
accuracy

91.10259684972328

In [54]:
y_test['is_promoted'] = Y_pred

In [55]:
y_test.to_csv('submission.csv', index=False)